In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import time
import csv
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.animation as animation
import matplotlib as mpl
from scipy import interpolate
from scipy import ndimage
#import numpy as np, cv2
import matplotlib.patches as patches

mpl.rcParams['axes.linewidth'] = 3.0 #set the value globally
#mpl.use("Agg")

In [3]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [4]:
import sys 
sys.path.insert(0, '/home/martial/Data/programs/Hyperspy_ungit')

import hyperspy.api as hs

In [5]:
hs.hyperspy.Release.info

'\n    H y p e r S p y\n    Version 1.6.dev\n\n    http://www.hyperspy.org\n\n    '

In [6]:
def align2D_cv(hs_stack, warp_mode):
    
    hs_stack.change_dtype('float32')

    #shift
    warp_matrix_max = np.zeros(4)
    
    # High pass filter using low pass and Gaussian blurring
    filter_1 = ndimage.gaussian_filter(hs_stack.data[0], 10)
    
    # Convert Numpy hpf data in openCV format (3 channels)
    img_1_hpf_cv = cv2.cvtColor(filter_1, cv2.COLOR_GRAY2BGR)
    # Convert openCV format (3 channels) in openCV format grey (1 channel)
    img_1_hpf_cv = cv2.cvtColor(img_1_hpf_cv, cv2.COLOR_BGR2GRAY)
     
    #find the shape
    sz = img_1_hpf_cv.shape
    
    #Translation ( MOTION_TRANSLATION ) : The first image can be shifted ( translated ) by (x , y) to obtain the second image. There are only two parameters x and y that we need to estimate.
    #Euclidean ( MOTION_EUCLIDEAN ) : The first image is a rotated and shifted version of the second image. So there are three parameters — x, y and angle . You will notice in Figure 4, when a square undergoes Euclidean transformation, the size does not change, parallel lines remain parallel, and right angles remain unchanged after transformation.
    #Affine ( MOTION_AFFINE ) : An affine transform is a combination of rotation, translation ( shift ), scale, and shear. This transform has six parameters. When a square undergoes an Affine transformation, parallel lines remain parallel, but lines meeting at right angles no longer remain orthogonal.
    #Homography ( MOTION_HOMOGRAPHY ) : All the transforms described above are 2D transforms. They do not account for 3D effects. A homography transform on the other hand can account for some 3D effects ( but not all ). This transform has 8 parameters. A square when transformed using a Homography can change to any quadrilateral.
    
    #warp_mode = cv2.MOTION_TRANSLATION
    
    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        
    warp_matrix_returned = np.empty((hs_stack.data.shape[0], warp_matrix.shape[0], warp_matrix.shape[1]))
    
    # Specify the number of iterations.
    number_of_iterations = 3000;
    
    # Specify the threshold of the increment
    termination_eps = 1e-10;
    
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    for i in range (1,hs_stack.data.shape[0]):
        filter_2 = ndimage.gaussian_filter(hs_stack.data[i], 10)
        # Convert Numpy hpf data in openCV format (3 channels)
        img_2_hpf_cv = cv2.cvtColor(filter_2, cv2.COLOR_GRAY2BGR)
        # Convert openCV format (3 channels) in openCV format grey (1 channel)
        img_2_hpf_cv = cv2.cvtColor(img_2_hpf_cv, cv2.COLOR_BGR2GRAY)
    
        # Run the ECC algorithm. The results are stored in warp_matrix.
        (cc, warp_matrix) = cv2.findTransformECC(img_1_hpf_cv,img_2_hpf_cv,warp_matrix, warp_mode, criteria)
    
        warp_matrix_returned[i,:,:] = warp_matrix
    
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            # Use warpPerspective for Homography 
            hs_stack.data[i] = cv2.warpPerspective (hs_stack.data[i], warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else :
            # Use warpAffine for Translation, Euclidean and Affine
            hs_stack.data[i] = cv2.warpAffine(hs_stack.data[i], warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP); 
           
        if warp_matrix[0,2]<warp_matrix_max[0]:
            warp_matrix_max[0]=warp_matrix[0,2]
            
        if warp_matrix[0,2]>warp_matrix_max[1]:
            warp_matrix_max[1]=warp_matrix[0,2]

        if warp_matrix[1,2]<warp_matrix_max[2]:
            warp_matrix_max[2]=warp_matrix[1,2]
            
        if warp_matrix[1,2]>warp_matrix_max[3]:
            warp_matrix_max[3]=warp_matrix[1,2]
            
        print(str(i) +' / '+str(hs_stack.data.shape[0]-1))
            
    hs_stack.crop_image(bottom=hs_stack.data.shape[2]-np.int(warp_matrix_max[3]), \
                        top = -np.int(warp_matrix_max[2]), \
                        left=-np.int(warp_matrix_max[0]), \
                        right = hs_stack.data.shape[1]-np.int(warp_matrix_max[1]))
    
    return warp_matrix_returned

In [7]:
def align2D_cv_only_align(hs_stack, warp_matrix, warp_mode):
    
    hs_stack.change_dtype('float32')

    #shift
    warp_matrix_max = np.zeros(4)
       
    #find the shape
    sz = hs_stack.data[0].shape
        
    for i in range (1,hs_stack.data.shape[0]):
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            # Use warpPerspective for Homography 
            hs_stack.data[i] = cv2.warpPerspective (hs_stack.data[i], warp_matrix[i,:,:], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else :
            # Use warpAffine for Translation, Euclidean and Affine
            hs_stack.data[i] = cv2.warpAffine(hs_stack.data[i], warp_matrix[i,:,:], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP); 
           
        if warp_matrix[i,0,2]<warp_matrix_max[0]:
            warp_matrix_max[0]=warp_matrix[i,0,2]
            
        if warp_matrix[i,0,2]>warp_matrix_max[1]:
            warp_matrix_max[1]=warp_matrix[i,0,2]

        if warp_matrix[i,1,2]<warp_matrix_max[2]:
            warp_matrix_max[2]=warp_matrix[i,1,2]
            
        if warp_matrix[i,1,2]>warp_matrix_max[3]:
            warp_matrix_max[3]=warp_matrix[i,1,2]
            
        print(str(i) +' / '+str(hs_stack.data.shape[0]-1))
 
    hs_stack.crop_image(bottom=hs_stack.data.shape[2]-np.int(warp_matrix_max[3]), \
                        top = -np.int(warp_matrix_max[2]), \
                        left=-np.int(warp_matrix_max[0]), \
                        right = hs_stack.data.shape[1]-np.int(warp_matrix_max[1]))
       
    return

### Alignement

In [ ]:
#Load dataset previously binned by 4
TEM_bin4 = hs.load('16)_380K_bin4_ali.dm4')

In [ ]:
#Select the area used to align the data
TEM_bin4.crop_image(top=80., left=20., bottom=115., right=100.)

In [ ]:
#Calculate the shift in between the images and return a matrix with the shifts
#It also aligns the TEM dataset according to the calculated shift
warp_matrix_returned_bin4 = align2D_cv(TEM_bin4, cv2.MOTION_TRANSLATION)

warp_matrix_returned_bin4 = np.load('warp_matrix_returned_bin4.npy')

np.save('warp_matrix_returned_bin4.npy',warp_matrix_returned_bin4)

In [ ]:
TEM_bin4 = hs.load('16)_380K_bin4_ali.dm4')

In [ ]:
#Align the data according to the wrap_matrix calculated previously
align2D_cv_only_align(TEM_bin4, warp_matrix_returned_bin4, cv2.MOTION_TRANSLATION)

In [ ]:
TEM_bin4.plot()

In [ ]:
#Calculate the swrp matrix for a different binned (2 instead of 4 in that case)
warp_matrix_returned_bin2 = np.empty_like(warp_matrix_returned_bin4)
warp_matrix_returned_bin2 = warp_matrix_returned_bin4
warp_matrix_returned_bin2[:,0,2] = 2*warp_matrix_returned_bin4[:,0,2]
warp_matrix_returned_bin2[:,1,2] = 2*warp_matrix_returned_bin4[:,1,2]

In [ ]:
TEM_bin2 = hs.load('16)_380K_bin2_ali.dm4')

In [ ]:
align2D_cv_only_align(TEM_bin2, warp_matrix_returned_bin2, cv2.MOTION_TRANSLATION)

In [ ]:
TEM_bin2.plot()

In [ ]:
TEM_bin2.save('TEM_bin2_aligned.hdf5')

In [ ]:
TEM_bin2.plot()